In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src.data_processing import AA_KEYS, compute_pfm, compute_ic_position, compute_ic, get_mia
from src.utils import pkl_dump, pkl_load
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 140
import warnings
warnings.filterwarnings("ignore")
from tqdm.autonotebook import tqdm
pd.set_option("display.max_columns",999)
pd.set_option("display.max_rows",100)
DATADIR = '../output_xls/'

C:\Users\richi\anaconda3\envs\phd\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hlas = np.unique([x.split('_')[0] for x in os.listdir(DATADIR) if 'chunk' in x and x.endswith('rank20.txt')])

In [4]:
for l in range(8,13):
    for hla in hlas:
        files = [os.path.join(DATADIR,x) for x in os.listdir(DATADIR) if hla in x and f'{l}' in x]
        df = pd.concat([pd.read_csv(x, sep = '\t', skiprows=1,header=0) for x in files])
        df1 = df.query('EL_Rank<0.1')
        df5 = df.query('EL_Rank<0.5')
        drop_idx1 = df1.loc[df1['Peptide'].apply(lambda x: any([z not in AA_KEYS for z in x]))].index
        drop_idx5 = df5.loc[df5['Peptide'].apply(lambda x: any([z not in AA_KEYS for z in x]))].index
        print(f'1: len\t{l}\t{hla}\t{len(df1.drop(index=drop_idx1).Peptide.values)}') 
        print(f'5: len\t{l}\t{hla}\t{len(df5.drop(index=drop_idx5).Peptide.values)}')

1: len	8	HLA-A0101	1193
5: len	8	HLA-A0101	10844
1: len	8	HLA-A0201	29
5: len	8	HLA-A0201	2475
1: len	8	HLA-A0206	0
5: len	8	HLA-A0206	610
1: len	8	HLA-A0301	40
5: len	8	HLA-A0301	1809
1: len	8	HLA-A1101	0
5: len	8	HLA-A1101	747
1: len	8	HLA-A2402	343
5: len	8	HLA-A2402	9891
1: len	8	HLA-B0702	5886
5: len	8	HLA-B0702	40786


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [5]:
hlas = np.unique([x.split('_')[0] for x in os.listdir(DATADIR) if 'chunk' in x and x.endswith('rank20.txt')])
lens = list(range(8,13))
results = {}
for l in tqdm(lens, desc='lens', leave=True, position=0):
    results[l]={}
    for hla in tqdm(hlas, desc='hlas', leave=True, position=1):
        files = [os.path.join(DATADIR,x) for x in os.listdir(DATADIR) if hla in x and f'{l}' in x]
        if len(files)==0:continue
        
        df = pd.concat([pd.read_csv(x, sep = '\t', skiprows=1,header=0) for x in files])
        df1 = df.query('EL_Rank<0.1')
        df5 = df.query('EL_Rank<0.5')
        drop_idx1 = df1.loc[df1['Peptide'].apply(lambda x: any([z not in AA_KEYS for z in x]))].index
        drop_idx5 = df5.loc[df5['Peptide'].apply(lambda x: any([z not in AA_KEYS for z in x]))].index
        
        seqs1 = df1.drop(index=drop_idx1).Peptide.values
        seqs5 = df5.drop(index=drop_idx5).Peptide.values
        
        results[l][hla]={}
        
        if len(seqs1)<1:
            results[l][hla]['0.5'] = compute_ic(seqs5)
        elif len(seqs1)>1:
            results[l][hla]['0.5'] = compute_ic(seqs5)
            results[l][hla]['0.1'] = compute_ic(seqs1)


lens: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [05:00<00:00, 60.11s/it]


In [26]:
results

{8: {'HLA-A0101': {'0.5': array([0.10426629, 0.32981354, 0.35519087, 0.05169463, 0.04807597,
          0.0540455 , 0.07578892, 0.97498005]),
   '0.1': array([0.13047594, 0.56412485, 0.6625151 , 0.06209439, 0.06002313,
          0.09028649, 0.09891242, 1.        ])},
  'HLA-A0201': {'0.5': array([0.30632842, 0.93592773, 0.15993166, 0.16441977, 0.19821942,
          0.18021345, 0.13964885, 0.72296146]),
   '0.1': array([0.72983083, 1.        , 0.59255168, 0.27502471, 0.37810522,
          0.41280013, 0.32925284, 0.76882029])},
  'HLA-A0206': {'0.5': array([0.44108713, 0.6601617 , 0.25473291, 0.21868807, 0.27714616,
          0.22662205, 0.19608277, 0.75832304])},
  'HLA-A0301': {'0.5': array([0.53085026, 0.54140276, 0.20032376, 0.18213725, 0.13866436,
          0.20711935, 0.08748633, 0.9738373 ]),
   '0.1': array([0.68123111, 0.67224383, 0.53653651, 0.44897366, 0.2898773 ,
          0.42832851, 0.21562797, 1.        ])},
  'HLA-A1101': {'0.5': array([0.45709258, 0.67600876, 0.19715565, 

In [7]:
pkl_dump(results, '../output/ic_results_dict.pkl')

../output/ic_results_dict.pkl saved.


In [8]:
results[8]['HLA-A0201']

{'0.5': array([0.30632842, 0.93592773, 0.15993166, 0.16441977, 0.19821942,
        0.18021345, 0.13964885, 0.72296146]),
 '0.1': array([0.72983083, 1.        , 0.59255168, 0.27502471, 0.37810522,
        0.41280013, 0.32925284, 0.76882029])}

In [4]:
results = pkl_load('../output/ic_results_dict.pkl')
for k1 in results.keys():
    for k2 in results[k1].keys():
        for thr in [0.125, 0.2, 0.3]:
            if type(results[k1][k2])==tuple:
                print(f'\tUsing rank 0.5\nlen:\t{k1}\tHLA:{k2}\tThreshold:{thr}:\n\tMIA:\t{get_mia(results[k1][k2][0], thr)}')
            else:
                print(f'len:\t{k1}\tHLA:{k2}\tThreshold:{thr}:\n\tMIA:\t{get_mia(results[k1][k2], thr)}')
            

len:	8	HLA:HLA-A0101	Threshold:0.125:
	MIA:	[3 4 5 6]
len:	8	HLA:HLA-A0101	Threshold:0.2:
	MIA:	[0 3 4 5 6]
len:	8	HLA:HLA-A0101	Threshold:0.3:
	MIA:	[0 3 4 5 6]
len:	8	HLA:HLA-A0201	Threshold:0.125:
	MIA:	[]
len:	8	HLA:HLA-A0201	Threshold:0.2:
	MIA:	[]
len:	8	HLA:HLA-A0201	Threshold:0.3:
	MIA:	[3]
	Using rank 0.5
len:	8	HLA:HLA-A0206	Threshold:0.125:
	MIA:	[]
	Using rank 0.5
len:	8	HLA:HLA-A0206	Threshold:0.2:
	MIA:	[6]
	Using rank 0.5
len:	8	HLA:HLA-A0206	Threshold:0.3:
	MIA:	[2 3 4 5 6]
len:	8	HLA:HLA-A0301	Threshold:0.125:
	MIA:	[]
len:	8	HLA:HLA-A0301	Threshold:0.2:
	MIA:	[]
len:	8	HLA:HLA-A0301	Threshold:0.3:
	MIA:	[4 6]
	Using rank 0.5
len:	8	HLA:HLA-A1101	Threshold:0.125:
	MIA:	[6]
	Using rank 0.5
len:	8	HLA:HLA-A1101	Threshold:0.2:
	MIA:	[2 4 6]
	Using rank 0.5
len:	8	HLA:HLA-A1101	Threshold:0.3:
	MIA:	[2 3 4 5 6]
len:	8	HLA:HLA-A2402	Threshold:0.125:
	MIA:	[]
len:	8	HLA:HLA-A2402	Threshold:0.2:
	MIA:	[2 3 4 5 6]
len:	8	HLA:HLA-A2402	Threshold:0.3:
	MIA:	[0 2 3 4 5 6]
len:	8	H